In [4]:
import pandas as pd
import numpy as np
import argparse
import py2neo as g


In [7]:
#G =  nx.read_edgelist()

# Lecture du fichier
fname ='./twitter_100M.csv'
twitter = pd.read_csv(fname,  header='infer',
        sep=' ',
        low_memory=False,
        dtype={'dest': np.int32, 'source': np.int32},
        nrows=1000)
print('heoo')
twitter.head(5)


heoo


,Source,Target
0,1,0
1,3,2
2,2,3
3,5,4
4,4,5


In [8]:
graph = Graph()

tx = graph.cypher.begin()
statement = """
                        MERGE (p:Person {twitterId: {twitterId}})
                        REMOVE p:Shadow
                        SET p.screenName = {screenName},
                            p.description = {description},
                            p.name = {name}
                        WITH p

                        FOREACH(followerId IN {followers} |
                          MERGE (follower:Person {twitterId: followerId})
                          ON CREATE SET follower:Shadow
                          MERGE (follower)-[:FOLLOWS]->(p)
                        )

                        FOREACH(friendId IN {friends} |
                          MERGE (friend:Person {twitterId: friendId})
                          ON CREATE SET friend:Shadow
                          MERGE (p)-[:FOLLOWS]->(friend)
                        )
                        """
tx.append(statement, params)
tx.process()    
tx.commit()

KeyError: 0